# Semantic SfM of single scene in Scannet

### 1a. Download entire Scannet dataset

Find information on ScanNet website: http://www.scan-net.org/ScanNet/

After download the data, I should extract data from .sens using [SensReader/reader.py](https://github.com/ScanNet/ScanNet/tree/master/SensReader/python). To extract all scans in the test data, I wrote a script to automate executing the python script, `reader.py`:

```python
import os
import subprocess

# Define the base directory for the scans_test
base_dir = "/home/zchen256/semantic_SfM/data/scannet/scans_test"

# Iterate over all files in the directory
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".sens"):
            # Construct the full path to the .sens file
            sens_file = os.path.join(root, file)
            # Construct the output path
            output_path = os.path.join(root, "output")
            # Create the output directory if it doesn't exist
            os.makedirs(output_path, exist_ok=True)
            # Construct the command
            command = [
                "python3", "reader.py",
                "--filename", sens_file,
                "--output_path", output_path,
                "--export_depth_images",
                "--export_color_images",
                "--export_poses",
                "--export_intrinsics"
            ]
            # Run the command
            print(command)
            subprocess.run(command)
```

### 1b. Extract Scannet data to prepare SSfM data in batch

In [3]:
from scene_extractor import *
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
scan_dir = '../../data/scannet/scans_test'
save_dir = '../../data/scannet/ssfm'
scene_dir = os.path.join(save_dir, 'scene0707_00')

In [5]:
# batch project
photos_folder_path = os.path.join(scene_dir, 'photos')
associations_folder_path = os.path.join(scene_dir, 'associations')
segmentations_folder_path = os.path.join(scene_dir, 'segmentations')
associations_folder_path = os.path.join(scene_dir, 'associations')

In [3]:
batch_extract_scenes(scan_dir, save_dir)

Extracted 5578 segmentations


### 2. Select keyimages based on blur

In [6]:
from select_keyimages import select_scannet_keyimages
import os

In [33]:
select_scannet_keyimages(scene_dir, ratio=0.5, threshold=150, file_cluster_size=5, file_select_window=5, n_jobs=8)

Processing images: 100%|██████████| 784/784 [00:04<00:00, 164.68it/s]


Number of keyimages_threshold:  5
Number of keyimages_ratio:  392
Number of keyimages:  392
Number of selected keyimages from window:  157
Number of selected keyimages:  244
Total images:  784
Keyimages saved to:  ../../data/scannet/ssfm/scene0707_00/associations/keyimages.yaml


### 3. Segment keyimages

In [8]:
from ssfm.image_segmentation import ImageSegmentation
import yaml

In [13]:
sam_params = {}
sam_params['model_name'] = 'sam'
sam_params['model_path'] = '../../semantic_SfM/sam/sam_vit_h_4b8939.pth'
sam_params['model_type'] = 'vit_h'
sam_params['device'] = 'cuda:3'
sam_params['points_per_side'] = 32
sam_params['pred_iou_thresh'] = 0.96
sam_params['stability_score_thresh'] = 0.96
sam_params['crop_n_layers'] = 1

In [9]:
using_keyimages = True

if using_keyimages:
    keyimages_path = os.path.join(scene_dir, 'associations', 'keyimages.yaml')
    # read keyimages
    with open(keyimages_path, 'r') as f:
        keyimages = yaml.load(f, Loader=yaml.FullLoader)

    # replace .npy with .jpg
    images = [keyimage.replace('.npy', '.jpg') for keyimage in keyimages]
    # sort images based on the values of keyimages in file names
    images = sorted(images, key=lambda x: int(x.split('_')[-1].split('.')[0]))
    image_paths = [os.path.join(scene_dir, 'photos', image) for image in images]

else:
    image_paths = [os.path.join(scene_dir, 'photos', image) for image in os.listdir(os.path.join(scene_dir, 'photos'))]
    # sort images based on the values of keyimages in file names
    image_paths = sorted(image_paths, key=lambda x: int(x.split('_')[-1].split('.')[0]))

print(image_paths)

['../../data/scannet/ssfm/scene0707_00/photos/0.jpg', '../../data/scannet/ssfm/scene0707_00/photos/2.jpg', '../../data/scannet/ssfm/scene0707_00/photos/5.jpg', '../../data/scannet/ssfm/scene0707_00/photos/6.jpg', '../../data/scannet/ssfm/scene0707_00/photos/7.jpg', '../../data/scannet/ssfm/scene0707_00/photos/11.jpg', '../../data/scannet/ssfm/scene0707_00/photos/14.jpg', '../../data/scannet/ssfm/scene0707_00/photos/15.jpg', '../../data/scannet/ssfm/scene0707_00/photos/19.jpg', '../../data/scannet/ssfm/scene0707_00/photos/21.jpg', '../../data/scannet/ssfm/scene0707_00/photos/23.jpg', '../../data/scannet/ssfm/scene0707_00/photos/24.jpg', '../../data/scannet/ssfm/scene0707_00/photos/25.jpg', '../../data/scannet/ssfm/scene0707_00/photos/29.jpg', '../../data/scannet/ssfm/scene0707_00/photos/31.jpg', '../../data/scannet/ssfm/scene0707_00/photos/32.jpg', '../../data/scannet/ssfm/scene0707_00/photos/35.jpg', '../../data/scannet/ssfm/scene0707_00/photos/39.jpg', '../../data/scannet/ssfm/scene07

In [35]:
image_segmentor = ImageSegmentation(sam_params)   
#image_segmentor.set_distortion_correction('../data/courtright/SfM_products/agisoft_cameras.xml')
segmentations_folder_path = os.path.join(scene_dir, 'segmentations')
image_segmentor.batch_predict(image_paths, segmentations_folder_path, maximum_size=10000, save_overlap=True)

Processing image 1/244.
Processing image 2/244.
Processing image 3/244.
Processing image 4/244.
Processing image 5/244.
Processing image 6/244.
Processing image 7/244.
Processing image 8/244.
Processing image 9/244.
Processing image 10/244.
Processing image 11/244.
Processing image 12/244.
Processing image 13/244.
Processing image 14/244.
Processing image 15/244.
Processing image 16/244.
Processing image 17/244.
Processing image 18/244.
Processing image 19/244.
Processing image 20/244.
Processing image 21/244.
Processing image 22/244.
Processing image 23/244.
Processing image 24/244.
Processing image 25/244.
Processing image 26/244.
Processing image 27/244.
Processing image 28/244.
Processing image 29/244.
Processing image 30/244.
Processing image 31/244.
Processing image 32/244.
Processing image 33/244.
Processing image 34/244.
Processing image 35/244.
Processing image 36/244.
Processing image 37/244.
Processing image 38/244.
Processing image 39/244.
Processing image 40/244.
Processin

### 4. Create projection associations

In [1]:
from ssfm.probabilistic_projection import *
import os
import yaml

In [11]:
pointcloud_projector = PointcloudProjection(depth_filtering_threshold=0.005)
pointcloud_projector.read_scannet_camera_parameters(scene_dir)
mesh_file_path = os.path.join(scene_dir, 'reconstructions', 'mesh_vertices_color.npy')
pointcloud_projector.read_scannet_mesh(mesh_file_path)

In [12]:
assert os.path.exists(keyimages_path), 'Keyimages not found'

with open(keyimages_path, 'r') as f:
    keyimages = yaml.safe_load(f)

# replace .npy with .jpg in keyimages, keyimages is a list of strings
image_list = [os.path.splitext(image)[0] + '.jpg' for image in keyimages]

In [10]:
#pointcloud_projector.parallel_batch_project(image_list, save_folder_path)
pointcloud_projector.parallel_batch_project_joblib(image_list, associations_folder_path, num_workers=16, save_depth=False)

Processing frames: 100%|██████████| 133/133 [00:07<00:00, 16.71it/s]


In [13]:
# build keyimage associations
from ssfm.keyimage_associations_builder import *

In [12]:
smc_solver = KeyimageAssociationsBuilder(associations_folder_path, segmentations_folder_path)
smc_solver.build_associations()

100%|██████████| 277/277 [00:01<00:00, 187.59it/s]


In [13]:
smc_solver.find_min_cover()

| Metric                                                       | Count      | Percentage           |
----------------------------------------------------------------------------------------------------
| Number of points not covered by any image                    | 12442      | 9.26                 |
| Number of points covered by less than or equal to 1 image    | 22628      | 16.84                |
| Number of points covered by less than or equal to 3 images   | 38236      | 28.46                |
| Number of points covered by less than or equal to 5 images   | 50937      | 37.91                |


### 5. Estimate memory usage

In [1]:
from ssfm.memory_calculator import memory_calculator

In [16]:
# pointcloud file
npy_file = mesh_file_path
# image file sample
image_file = os.path.join(photos_folder_path, image_list[0])
# number of images
num_images = len(image_list)
# number of segmentation ids for each point in the point cloud
num_segmentation_ids = 5

memory_calculator(npy_file, image_file, num_images, num_segmentation_ids)

+----------------------------------------+-----------------------+
|              Memory Type               |  Memory Required (GB) |
+----------------------------------------+-----------------------+
|      Segmentation for each image       |  0.002336740493774414 |
| Pixel2point association for each image |  0.004673480987548828 |
| Point2pixel association for each image | 0.0005005300045013428 |
|                                        |                       |
|      Segmentation for all images       |   4.673480987548828   |
| Pixel2point association for all images |   9.346961975097656   |
| Point2pixel association for all images |   1.0010600090026855  |
|          pc_segmentation_ids           |  0.002502650022506714 |
|         pc_segmentation_probs          |  0.002502650022506714 |
|          keyimage_association          |   0.2502650022506714  |
|                 Total                  |   15.276773273944855  |
+----------------------------------------+--------------------

### 6. Run object registration

In [1]:
from ssfm.object_registration import *
from ssfm.post_processing import *

In [7]:
pointcloud_path = os.path.join(scene_dir, 'reconstructions', 'mesh_vertices_color.npy')
segmentations_folder_path = os.path.join(scene_dir, 'segmentations')

keyimage_associations_file_name = 'associations_keyimage.npy'
keyimage_yaml_name = 'keyimages.yaml'

In [48]:
# Create object registration
obr = ObjectRegistration(pointcloud_path, segmentations_folder_path, associations_folder_path, keyimage_associations_file_name=keyimage_associations_file_name, keyimage_yaml_name=keyimage_yaml_name)

# Run object registration
obr.object_registration(iou_threshold=0.5, save_semantics=True)


# save semantic point cloud
save_path = os.path.join(scene_dir, 'semantic_model.las')
obr.save_semantic_pointcloud(save_path)

Processing image 1/244: 0.npy
Processing image 2/244: 100.npy
Processing image 3/244: 104.npy
Processing image 4/244: 105.npy
Processing image 5/244: 109.npy
Processing image 6/244: 11.npy
Processing image 7/244: 111.npy
Processing image 8/244: 112.npy
Processing image 9/244: 115.npy
Processing image 10/244: 117.npy
Processing image 11/244: 121.npy
Processing image 12/244: 122.npy
Processing image 13/244: 126.npy
Processing image 14/244: 128.npy
Processing image 15/244: 130.npy
Processing image 16/244: 134.npy
Processing image 17/244: 135.npy
Processing image 18/244: 137.npy
Processing image 19/244: 14.npy
Processing image 20/244: 141.npy
Processing image 21/244: 142.npy
Processing image 22/244: 145.npy
Processing image 23/244: 149.npy
Processing image 24/244: 15.npy
Processing image 25/244: 150.npy
Processing image 26/244: 152.npy
Processing image 27/244: 155.npy
Processing image 28/244: 159.npy
Processing image 29/244: 161.npy
Processing image 30/244: 163.npy
Processing image 31/244:

In [8]:
image_id = 243
semantics_folder_path = os.path.join(associations_folder_path, 'semantics', 'semantics_{}.npy'.format(image_id))
save_las_path = os.path.join(associations_folder_path, 'semantics', 'semantics_{}.las'.format(image_id))
add_semantics_to_pointcloud(pointcloud_path, semantics_folder_path, save_las_path) 

maximum of semantics:  8558
number of unique semantics:  389


In [9]:
semantic_pc_file_path = save_las_path
post_processing = PostProcessing(semantic_pc_file_path)
post_processing.shuffle_semantic_ids()
save_las_path = os.path.join(associations_folder_path, 'semantics', 'semantics_{}_shuffled.las'.format(image_id))
post_processing.save_semantic_pointcloud(save_las_path)

Number of unique semantics:  389
